In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Connect to R

In [ ]:
!pip install rpy2

In [ ]:
import rpy2
from rpy2 import robjects

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")


Update all/some/none? [a/s/n]: n


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.26.tar.gz'
Content type 'application/x-gzip' length 594489 bytes (580 KB)
downloaded 580 KB


The downloaded source packages are in
	‘/tmp/RtmpkWgkVa/downloaded_packages’
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 'BiocVersion', 'DESeq2'
also installing the dependencies ‘formatR’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘SparseArray’, ‘lambda.r’, ‘futile.options’, ‘GenomeInfoDb’, ‘XVector’, ‘S4Arrays’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘BiocGenerics’, ‘Biobase’, ‘BiocParallel’, ‘matrixStats’, ‘locfit’, ‘MatrixGenerics’, ‘Rcp

In [ ]:
%%R

install.packages("readr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/readr_2.1.5.tar.gz'
Content type 'application/x-gzip' length 298065 bytes (291 KB)
downloaded 291 KB


The downloaded source packages are in
	‘/tmp/RtmpkWgkVa/downloaded_packages’


In [ ]:
%%R

setwd("/content/drive/MyDrive/Tugas Akhir/preprocessing_data")

In [ ]:
%%R

library(dplyr)
library(tidyverse)
library(readr)
library(DESeq2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.2     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.1.0     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: generics

Attaching package: ‘generics’

The following object is masked from ‘package:lubridate’:

    as.difftime

The following object is masked from ‘package:dplyr’:

    explain

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union


Attaching package: ‘BiocGenerics’

The following object is masked from ‘package:dplyr’:

    combine

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.c

## Load Data

In [ ]:
%%R

df1 = read_tsv("train_raw_data.tsv")
head(df1)

New names:
• `` -> `...1`
Rows: 76766 Columns: 1853
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr    (1): ...1
dbl (1852): GSM7461809, MP2PRT_ALL_482, GSE277354_67, MP2PRT_ALL_1066, MP2PR...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 1,853
  ...1     GSM7461809 MP2PRT_ALL_482 GSE277354_67 MP2PRT_ALL_1066 MP2PRT_ALL_613
  <chr>         <dbl>          <dbl>        <dbl>           <dbl>          <dbl>
1 5S_rRNA           0              0            0               0              0
2 5_8S_rR…          0              0            0               0              0
3 7SK               0              0            0               0              0
4 A1BG             59            139            8             175             21
5 A1BG-AS1         27            179           15             291              0
6 A1CF    

In [ ]:
%%R

#df_count <- df1 %>% remove_rownames %>% column_to_rownames(var="patient_ID")
#head(df_count)

          MP2PRT_ALL_577 TCGA_LAML_4 MP2PRT_ALL_500 GSM6068980 MP2PRT_ALL_552
5_8S_rRNA              0           0              0          0              0
7SK                    0           0              0          0              0
A1BG                  10          44            534         59            102
A1BG-AS1              87          41            431        133            201
A1CF                   7           0              2          7              1
A2M                  261         445             41        388            125
          MP2PRT_ALL_565 GSE277354_34 MP2PRT_ALL_324 MP2PRT_ALL_47 GSM6340874
5_8S_rRNA              0            0              0             0          0
7SK                    0            0              0             0          0
A1BG                 153            3             78           306         37
A1BG-AS1             401            9             88           341         16
A1CF                   0            0              0            

In [ ]:
%%R

df1_temp <- df1 %>% remove_rownames %>% column_to_rownames(var="...1")
head(df1_temp)

          GSM7461809 MP2PRT_ALL_482 GSE277354_67 MP2PRT_ALL_1066 MP2PRT_ALL_613
5S_rRNA            0              0            0               0              0
5_8S_rRNA          0              0            0               0              0
7SK                0              0            0               0              0
A1BG              59            139            8             175             21
A1BG-AS1          27            179           15             291              0
A1CF               0              0            0               2            622
          GSM3028297 GSE277354_65 MP2PRT_ALL_1005 TCGA_LAML_61 MP2PRT_ALL_1131
5S_rRNA            0            0               0            0               0
5_8S_rRNA          0            0               0            0               0
7SK                0            0               0            0               0
A1BG             215           14             252          109              88
A1BG-AS1          12           32            

In [ ]:
%%R

df_count <- df1_temp %>% select(-...1)
head(df_count)

Error in `select()`:
! Can't select columns that don't exist.
✖ Column `...1` doesn't exist.
Run `rlang::last_trace()` to see where the error occurred.

Error in select(., -...1) :


RInterpreterError: Failed to parse and evaluate line '\ndf_count <- df1_temp %>% select(-...1)\nhead(df_count)\n'.
R error message: 'Error in select(., -...1) :'

In [ ]:
%%R

df2 <- read_tsv("training_info.tsv")
head(df2)

Rows: 1852 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): patient_ID, class
dbl (1): batch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 3
  patient_ID      class        batch
  <chr>           <chr>        <dbl>
1 GSM7461809      non-leukemia    29
2 MP2PRT_ALL_482  ALL             38
3 GSE277354_67    CML             33
4 MP2PRT_ALL_1066 ALL             38
5 MP2PRT_ALL_613  ALL             38
6 GSM3028297      non-leukemia     8


In [ ]:
%%R

label <- df2 %>% remove_rownames %>% column_to_rownames(var='patient_ID')
head(label)

                       class batch
GSM7461809      non-leukemia    29
MP2PRT_ALL_482           ALL    38
GSE277354_67             CML    33
MP2PRT_ALL_1066          ALL    38
MP2PRT_ALL_613           ALL    38
GSM3028297      non-leukemia     8


In [ ]:
%%R

label <- df2 %>% select(-...1)
label <- label %>% remove_rownames %>% column_to_rownames(var='patient_ID')
head(label)

                  class batch
GSM1256809 non-leukemia     0
GSM1256810 non-leukemia     0
GSM1256811 non-leukemia     0
GSM1256812 non-leukemia     0
GSM1256813 non-leukemia     0
GSM1256814 non-leukemia     0


In [ ]:
%%R
label <- label[1]
head(label)

                       class
GSM7461809      non-leukemia
MP2PRT_ALL_482           ALL
GSE277354_67             CML
MP2PRT_ALL_1066          ALL
MP2PRT_ALL_613           ALL
GSM3028297      non-leukemia


In [ ]:
%%R

nrow(label)

[1] 1852


In [ ]:
%%R

ncol(df1_temp)

[1] 1852


In [ ]:
%%R

all(colnames(df1_temp) %in% rownames(label))

[1] TRUE


In [ ]:
%%R

all(colnames(df1_temp) == rownames(label))

[1] TRUE


## Deseq2

In [ ]:
%%R

dds <- DESeqDataSetFromMatrix(countData = df1_temp,
                       colData = label,
                       design = ~ class)

converting counts to integer mode
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
In addition: Warning message:
In DESeqDataSet(se, design = design, ignoreRank) :
  some variables in design formula are characters, converting to factors


In [ ]:
%%R

print(dds)

class: DESeqDataSet 
dim: 76766 1852 
metadata(1): version
assays(1): counts
rownames(76766): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(0):
colnames(1852): GSM7461809 MP2PRT_ALL_482 ... MP2PRT_ALL_339
  MP2PRT_ALL_779
colData names(1): class


In [ ]:
%%R
dds <- estimateSizeFactors(dds)
norm_counts <- counts(dds, normalized = TRUE)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R
head(norm_counts)

          GSM7461809 MP2PRT_ALL_482 GSE277354_67 MP2PRT_ALL_1066 MP2PRT_ALL_613
5S_rRNA       0.0000         0.0000      0.00000        0.000000        0.00000
5_8S_rRNA     0.0000         0.0000      0.00000        0.000000        0.00000
7SK           0.0000         0.0000      0.00000        0.000000        0.00000
A1BG        929.4896       116.1382     21.82444      100.493185       20.71686
A1BG-AS1    425.3597       149.5593     40.92082      167.105811        0.00000
A1CF          0.0000         0.0000      0.00000        1.148494      613.61357
          GSM3028297 GSE277354_65 MP2PRT_ALL_1005 TCGA_LAML_61 MP2PRT_ALL_1131
5S_rRNA      0.00000     0.000000        0.000000     0.000000        0.000000
5_8S_rRNA    0.00000     0.000000        0.000000     0.000000        0.000000
7SK          0.00000     0.000000        0.000000     0.000000        0.000000
A1BG      1144.17593    35.718311      335.241971   106.044894      114.949481
A1BG-AS1    63.86098    81.641855      359.18

In [ ]:
%%R
#write.table(norm_counts, file = "count_normalized_only.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

In [ ]:
%%R
dds <- estimateSizeFactors(dds)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


### Data Training

In [ ]:
%%R
dds_train <- estimateSizeFactors(dds)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R

dds_train <- estimateDispersions(dds_train)

gene-wise dispersion estimates
mean-dispersion relationship
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
final dispersion estimates


In [ ]:
%%R

disp_func <- dispersionFunction(dds_train)

In [ ]:
%%R
dds_train

class: DESeqDataSet 
dim: 76766 1852 
metadata(1): version
assays(2): counts mu
rownames(76766): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(10): baseMean baseVar ... dispOutlier dispMAP
colnames(1852): GSM7461809 MP2PRT_ALL_482 ... MP2PRT_ALL_339
  MP2PRT_ALL_779
colData names(2): class sizeFactor


In [ ]:
%%R
vsd_train <- varianceStabilizingTransformation(dds_train, blind = FALSE)

In [ ]:
%%R
normalize_train <- assay(vsd_train)

In [ ]:
%%R
head(normalize_train)

          GSM7461809 MP2PRT_ALL_482 GSE277354_67 MP2PRT_ALL_1066 MP2PRT_ALL_613
5S_rRNA     1.948727       1.948727     1.948727        1.948727       1.948727
5_8S_rRNA   1.948727       1.948727     1.948727        1.948727       1.948727
7SK         1.948727       1.948727     1.948727        1.948727       1.948727
A1BG        9.872205       6.951150     4.859562        6.755902       4.802372
A1BG-AS1    8.758376       7.296321     5.594942        7.449076       1.948727
A1CF        1.948727       1.948727     1.948727        2.726197       9.279170
          GSM3028297 GSE277354_65 MP2PRT_ALL_1005 TCGA_LAML_61 MP2PRT_ALL_1131
5S_rRNA     1.948727     1.948727        1.948727     1.948727        1.948727
5_8S_rRNA   1.948727     1.948727        1.948727     1.948727        1.948727
7SK         1.948727     1.948727        1.948727     1.948727        1.948727
A1BG       10.169779     5.429387        8.421725     6.828256        6.937210
A1BG-AS1    6.157304     6.478898        8.51

In [ ]:
%%R
#write.table(normalize_train, file = "normalized_train.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

## Data Validation

### Load Data

In [ ]:
%%R

df2 = read_tsv("validation_raw_data.tsv")
head(df2)

New names:
• `` -> `...1`
Rows: 76766 Columns: 524
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): ...1
dbl (523): GSM1614669, MP2PRT_ALL_1024, MP2PRT_ALL_1241, MP2PRT_ALL_337, MP2...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 524
  ...1  GSM1614669 MP2PRT_ALL_1024 MP2PRT_ALL_1241 MP2PRT_ALL_337 MP2PRT_ALL_161
  <chr>      <dbl>           <dbl>           <dbl>          <dbl>          <dbl>
1 5S_r…          0               0               0              0              0
2 5_8S…          0               0               0              0              0
3 7SK            0               1               0              0              0
4 A1BG         367             239              63            129            124
5 A1BG…        233             324             202            228            143
6 A1CF        

In [ ]:
%%R

df2_temp <- df2 %>% remove_rownames %>% column_to_rownames(var="...1")
head(df2_temp)

          GSM1614669 MP2PRT_ALL_1024 MP2PRT_ALL_1241 MP2PRT_ALL_337
5S_rRNA            0               0               0              0
5_8S_rRNA          0               0               0              0
7SK                0               1               0              0
A1BG             367             239              63            129
A1BG-AS1         233             324             202            228
A1CF               3              21               2              1
          MP2PRT_ALL_161 MP2PRT_ALL_118 GSM1256824 TCGA_LAML_86 GSM7968123
5S_rRNA                0              0          0            0          0
5_8S_rRNA              0              0          0            0          0
7SK                    0              0          0            0          0
A1BG                 124             63        455           11        238
A1BG-AS1             143            243        246           79        158
A1CF                   3              0          0            1         11

In [ ]:
%%R

df2_label <- read_tsv("validation_info.tsv")
head(df2_label)

Rows: 523 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): patient_ID, class
dbl (1): batch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 3
  patient_ID      class batch
  <chr>           <chr> <dbl>
1 GSM1614669      CLL       2
2 MP2PRT_ALL_1024 ALL      38
3 MP2PRT_ALL_1241 ALL      38
4 MP2PRT_ALL_337  ALL      38
5 MP2PRT_ALL_161  ALL      38
6 MP2PRT_ALL_118  ALL      38


In [ ]:
%%R

label_val <- df2_label %>% remove_rownames %>% column_to_rownames(var='patient_ID')
head(label_val)

                class batch
GSM1614669        CLL     2
MP2PRT_ALL_1024   ALL    38
MP2PRT_ALL_1241   ALL    38
MP2PRT_ALL_337    ALL    38
MP2PRT_ALL_161    ALL    38
MP2PRT_ALL_118    ALL    38


In [ ]:
%%R
label_val <- label_val[1]
head(label_val)

                class
GSM1614669        CLL
MP2PRT_ALL_1024   ALL
MP2PRT_ALL_1241   ALL
MP2PRT_ALL_337    ALL
MP2PRT_ALL_161    ALL
MP2PRT_ALL_118    ALL


In [ ]:
%%R
nrow(label_val)

[1] 523


In [ ]:
%%R
ncol(df2_temp)

[1] 523


In [ ]:
%%R

all(colnames(df2_temp) %in% rownames(label_val))

[1] TRUE


In [ ]:
%%R

all(colnames(df2_temp) == rownames(label_val))

[1] TRUE


### DESeq2

In [ ]:
%%R

dds_val <- DESeqDataSetFromMatrix(countData = df2_temp,
                       colData = label_val,
                       design = ~ class)

converting counts to integer mode
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
In addition: Warning message:
In DESeqDataSet(se, design = design, ignoreRank) :
  some variables in design formula are characters, converting to factors


In [ ]:
%%R
print(dds_val)

class: DESeqDataSet 
dim: 76766 523 
metadata(1): version
assays(1): counts
rownames(76766): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(0):
colnames(523): GSM1614669 MP2PRT_ALL_1024 ... MP2PRT_ALL_30
  MP2PRT_ALL_1218
colData names(1): class


In [ ]:
%%R
dds_val <- estimateSizeFactors(dds_val)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R
dispersionFunction(dds_val) <- disp_func

variance of dispersion residuals not estimated (necessary only for differential expression calling)
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R
vsd_val <- varianceStabilizingTransformation(dds_val, blind = FALSE)

In [ ]:
%%R
normalize_val <- assay(vsd_val)

In [ ]:
%%R
head(normalize_val)

          GSM1614669 MP2PRT_ALL_1024 MP2PRT_ALL_1241 MP2PRT_ALL_337
5S_rRNA     1.948727        1.948727        1.948727       1.948727
5_8S_rRNA   1.948727        1.948727        1.948727       1.948727
7SK         1.948727        2.529530        1.948727       1.948727
A1BG        8.364771        7.314595        5.446242       6.828532
A1BG-AS1    7.728341        7.735490        6.951127       7.608150
A1CF        3.108480        4.343682        2.726649       2.608774
          MP2PRT_ALL_161 MP2PRT_ALL_118 GSM1256824 TCGA_LAML_86 GSM7968123
5S_rRNA         1.948727       1.948727   1.948727     1.948727   1.948727
5_8S_rRNA       1.948727       1.948727   1.948727     1.948727   1.948727
7SK             1.948727       1.948727   1.948727     1.948727   1.948727
A1BG            6.758977       5.309624   8.292195     3.857451   7.410835
A1BG-AS1        6.951369       7.047157   7.434360     5.989127   6.851680
A1CF            3.066781       1.948727   1.948727     2.562491   3.826001

In [ ]:
%%R
#write.table(normalize_val, file = "normalized_validation.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

## Testing Data

### Load Data

In [ ]:
%%R

df3 = read_tsv("testing_raw_data.tsv")
head(df3)

New names:
• `` -> `...1`
Rows: 76766 Columns: 264
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): ...1
dbl (263): 7653-HSC, GSM7461821, 7256-Mono, GSM1554643, MP2PRT_ALL_818, TCGA...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 264
  ...1  `7653-HSC` GSM7461821 `7256-Mono` GSM1554643 MP2PRT_ALL_818 TCGA_LAML_97
  <chr>      <dbl>      <dbl>       <dbl>      <dbl>          <dbl>        <dbl>
1 5S_r…          0          0           0          0              0            0
2 5_8S…          0          0           0          0              0            0
3 7SK            0          0           0          0              0            0
4 A1BG           5         34           1        121             84          109
5 A1BG…          0         37           1         73            155          159
6 A1CF        

In [ ]:
%%R

df3_temp <- df3 %>% remove_rownames %>% column_to_rownames(var="...1")
head(df3_temp)

          7653-HSC GSM7461821 7256-Mono GSM1554643 MP2PRT_ALL_818 TCGA_LAML_97
5S_rRNA          0          0         0          0              0            0
5_8S_rRNA        0          0         0          0              0            0
7SK              0          0         0          0              0            0
A1BG             5         34         1        121             84          109
A1BG-AS1         0         37         1         73            155          159
A1CF             0          0         0          2              0            3
          GSM3028278 TCGA_LAML_132 MP2PRT_ALL_1280 MP2PRT_ALL_876
5S_rRNA            0             0               0              2
5_8S_rRNA          0             0               0              0
7SK                0             0               0              0
A1BG             143            55             514            190
A1BG-AS1          36            86             454            272
A1CF               1             2               9 

In [ ]:
%%R

df3_label <- read_tsv("test_info.tsv")
head(df3_label)

Rows: 263 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): patient_ID, class
dbl (1): batch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 3
  patient_ID     class        batch
  <chr>          <chr>        <dbl>
1 7653-HSC       non-leukemia     3
2 GSM7461821     non-leukemia    29
3 7256-Mono      non-leukemia     3
4 GSM1554643     AML              1
5 MP2PRT_ALL_818 ALL             38
6 TCGA_LAML_97   AML             37


In [ ]:
%%R

label_test <- df3_label %>% remove_rownames %>% column_to_rownames(var='patient_ID')
head(label_test)

                      class batch
7653-HSC       non-leukemia     3
GSM7461821     non-leukemia    29
7256-Mono      non-leukemia     3
GSM1554643              AML     1
MP2PRT_ALL_818          ALL    38
TCGA_LAML_97            AML    37


In [ ]:
%%R
label_test <- label_test[1]
head(label_test)

                      class
7653-HSC       non-leukemia
GSM7461821     non-leukemia
7256-Mono      non-leukemia
GSM1554643              AML
MP2PRT_ALL_818          ALL
TCGA_LAML_97            AML


In [ ]:
%%R
nrow(label_test)

[1] 263


In [ ]:
%%R
ncol(df3_temp)

[1] 263


In [ ]:
%%R

all(colnames(df3_temp) %in% rownames(label_test))

[1] TRUE


In [ ]:
%%R

all(colnames(df3_temp) == rownames(label_test))

[1] TRUE


### DESeq2

In [ ]:
%%R

dds_test <- DESeqDataSetFromMatrix(countData = df3_temp,
                       colData = label_test,
                       design = ~ class)

converting counts to integer mode
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
In addition: Warning message:
In DESeqDataSet(se, design = design, ignoreRank) :
  some variables in design formula are characters, converting to factors


In [ ]:
%%R
print(dds_test)

class: DESeqDataSet 
dim: 76766 263 
metadata(1): version
assays(1): counts
rownames(76766): 5S_rRNA 5_8S_rRNA ... hsa-mir-423 snoZ196
rowData names(0):
colnames(263): 7653-HSC GSM7461821 ... GSM1554629 GSM4280633
colData names(1): class


In [ ]:
%%R
dds_test <- estimateSizeFactors(dds_test)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R
dispersionFunction(dds_test) <- disp_func

variance of dispersion residuals not estimated (necessary only for differential expression calling)
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]


In [ ]:
%%R
vsd_test <- varianceStabilizingTransformation(dds_test, blind = FALSE)

In [ ]:
%%R
normalize_test <- assay(vsd_test)

In [ ]:
%%R
head(normalize_test)

          7653-HSC GSM7461821 7256-Mono GSM1554643 MP2PRT_ALL_818 TCGA_LAML_97
5S_rRNA   1.948727   1.948727  1.948727   1.948727       1.948727     1.948727
5_8S_rRNA 1.948727   1.948727  1.948727   1.948727       1.948727     1.948727
7SK       1.948727   1.948727  1.948727   1.948727       1.948727     1.948727
A1BG      5.229012   8.601156  3.756360   7.725398       6.278116     7.008271
A1BG-AS1  1.948727   8.720841  3.756360   7.029768       7.097223     7.526190
A1CF      1.948727   1.948727  1.948727   3.252208       1.948727     3.245360
          GSM3028278 TCGA_LAML_132 MP2PRT_ALL_1280 MP2PRT_ALL_876
5S_rRNA     1.948727      1.948727        1.948727       2.794271
5_8S_rRNA   1.948727      1.948727        1.948727       1.948727
7SK         1.948727      1.948727        1.948727       1.948727
A1BG        8.710617      5.782573        9.100684       7.100463
A1BG-AS1    6.795982      6.356886        8.924267       7.593976
A1CF        3.158230      2.897545        4.025365 

In [ ]:
%%R
#write.table(normalize_test, file = "normalized_testing.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

##DLL

In [ ]:
%%R
vsd <- vst(dds)

Exception ignored from cffi callback <function _processevents at 0x7a491c174ae0>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rpy2/rinterface_lib/callbacks.py", line 283, in _processevents
    @ffi_proxy.callback(ffi_proxy._processevents_def,
    
KeyboardInterrupt: 
Exception ignored from cffi callback <function _processevents at 0x7a491c174ae0>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rpy2/rinterface_lib/callbacks.py", line 283, in _processevents
    @ffi_proxy.callback(ffi_proxy._processevents_def,
    
KeyboardInterrupt: 


In [ ]:
%%R
normalize_data <- assay(vsd)
head(normalize_data)

          GSM1256809 GSM1256810 GSM1256811 GSM1256812 GSM1256813 GSM1256814
5S_rRNA     2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
5_8S_rRNA   2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
7SK         2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
A1BG        6.969373   6.425859   7.289138   7.286760   8.310029   8.560355
A1BG-AS1    6.834064   6.174882   6.754725   7.677744   7.922280   8.241491
A1CF        3.830472   2.795913   2.795913   2.795913   3.772782   3.350590
          GSM1256815 GSM1256816 GSM1256817 GSM1256818 GSM1256819 GSM1256820
5S_rRNA     2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
5_8S_rRNA   2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
7SK         2.795913   2.795913   2.795913   2.795913   2.795913   2.795913
A1BG        7.857687   8.599797   4.234439   4.392051   5.427197   4.318556
A1BG-AS1    7.715837   8.161871   4.760319   4.194394   5.026004   3.995958
A1CF        

In [ ]:
%%R
#write.table(normalize_data, file = "normalized_data.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

In [ ]:
%%R
#sizeFactors(dds)

     GSM1256809      GSM1256810      GSM1256811      GSM1256812      GSM1256813 
     0.80495978      0.66072615      0.99423498      0.65934219      1.51168760 
     GSM1256814      GSM1256815      GSM1256816      GSM1256817      GSM1256818 
     0.96221489      1.56685730      0.68946377      0.40021374      0.42536207 
     GSM1256819      GSM1256820      GSM1256821      GSM1256822      GSM1256823 
     0.49571587      0.58955373      0.39752496      0.90860064      0.90625304 
     GSM1256824      GSM1256825      GSM1256826      GSM1256827      GSM1256828 
     1.40387712      0.99875657      0.86149806      1.25316091      1.06637139 
     GSM1256829      GSM1256830      GSM1256831      GSM1256832      GSM1554577 
     1.15740787      1.52021474      1.17545860      1.08354990      0.34516214 
     GSM1554578      GSM1554579      GSM1554580      GSM1554581      GSM1554582 
     0.43494850      0.40415598      0.59033384      0.55352099      0.30159073 
     GSM1554583      GSM1554

simpan size factor untuk data validasi dan testing

In [ ]:
%%R
train_size_factors <- sizeFactors(dds)

In [ ]:
%%R
normalized_train_counts <- counts(dds, normalized=TRUE)

In [ ]:
%%R
#write.table(normalized_train_counts, file = "data_training_normalized.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

In [ ]:
%%R
df_check = read_tsv("data_training_normalized.tsv")
head(df_check)

New names:
• `` -> `...1`
Rows: 76765 Columns: 1853
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr    (1): ...1
dbl (1852): MP2PRT_ALL_577, TCGA_LAML_4, MP2PRT_ALL_500, GSM6068980, MP2PRT_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 1,853
  ...1      MP2PRT_ALL_577 TCGA_LAML_4 MP2PRT_ALL_500 GSM6068980 MP2PRT_ALL_552
  <chr>              <dbl>       <dbl>          <dbl>      <dbl>          <dbl>
1 5_8S_rRNA           0            0             0          0             0    
2 7SK                 0            0             0          0             0    
3 A1BG                8.35        30.6         480.        29.3          93.2  
4 A1BG-AS1           72.7         28.5         387.        66.1         184.   
5 A1CF                5.85         0             1.80       3.48          0.914
6 A2M            

##Normalization Validation and Training Data

In [ ]:
%%R
apply_normalization <- function(new_counts, sf_reference){
    normalized <- t(t(new_counts) / sf_reference)
    return(normalized)
}

### Validation Data

In [ ]:
%%R

df_val <- read_tsv("validation_dataset.tsv")
head(df_val)

Rows: 76765 Columns: 524
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): patient_ID
dbl (523): MP2PRT_ALL_1056, GSM2433722, GSM1554597, MP2PRT_ALL_60, GSM746180...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 524
  patient_ID MP2PRT_ALL_1056 GSM2433722 GSM1554597 MP2PRT_ALL_60 GSM7461803
  <chr>                <dbl>      <dbl>      <dbl>         <dbl>      <dbl>
1 5_8S_rRNA                0          0          0             0          0
2 7SK                      0          0          0             0          0
3 A1BG                   140        147         92            81         68
4 A1BG-AS1               309         81         84           201         65
5 A1CF                    20          0          5             9          0
6 A2M                    376         66        219          1230     

In [ ]:
%%R
valid_count <- df_val %>% remove_rownames %>% column_to_rownames(var="patient_ID")
head(valid_count)

          MP2PRT_ALL_1056 GSM2433722 GSM1554597 MP2PRT_ALL_60 GSM7461803
5_8S_rRNA               0          0          0             0          0
7SK                     0          0          0             0          0
A1BG                  140        147         92            81         68
A1BG-AS1              309         81         84           201         65
A1CF                   20          0          5             9          0
A2M                   376         66        219          1230         37
          GSM3104740 MP2PRT_ALL_985 MP2PRT_ALL_293 MP2PRT_ALL_168 GSM6062702
5_8S_rRNA          0              0              0              0          0
7SK                0              0              0              0          0
A1BG             514             99             45            182          1
A1BG-AS1         290            231            123            461          0
A1CF               5              7              4              3          0
A2M                3       

In [ ]:
%%R
normalized_val <- apply_normalization(valid_count, train_size_factors)

In addition: Warning message:
In t(new_counts)/sf_reference :
  longer object length is not a multiple of shorter object length


In [ ]:
%%R
head(normalized_val)

          MP2PRT_ALL_1056 GSM2433722  GSM1554597 MP2PRT_ALL_60 GSM7461803
5_8S_rRNA         0.00000    0.00000    0.000000       0.00000    0.00000
7SK               0.00000    0.00000    0.000000       0.00000    0.00000
A1BG            291.26164  112.78322   94.126946     160.79308   30.46598
A1BG-AS1       1048.59003  833.48180 1207.832417     140.24512  191.87126
A1CF             13.23878    0.00000    3.060283       4.73999    0.00000
A2M             127.91424   43.95472 1743.466682     862.17685   41.15538
          GSM3104740 MP2PRT_ALL_985 MP2PRT_ALL_293 MP2PRT_ALL_168 GSM6062702
5_8S_rRNA   0.000000       0.000000        0.00000       0.000000  0.0000000
7SK         0.000000       0.000000        0.00000       0.000000  0.0000000
A1BG      416.648950     123.800383       30.06210     120.630483  0.6075146
A1BG-AS1  769.362355     205.663283       79.27275     589.605729  0.0000000
A1CF       11.839097       4.670252        2.95973       1.684628  0.0000000
A2M         2.198011

In [ ]:
%%R
l_valid <- read_tsv("validation_labels.tsv")
head(l_valid)

Rows: 523 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): patient_ID, class
dbl (1): batch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 3
  patient_ID      class        batch
  <chr>           <chr>        <dbl>
1 MP2PRT_ALL_1056 ALL             38
2 GSM2433722      CLL              4
3 GSM1554597      AML              1
4 MP2PRT_ALL_60   ALL             38
5 GSM7461803      non-leukemia    29
6 GSM3104740      CLL             10


In [ ]:
%%R
valid_label <- l_valid %>% remove_rownames %>% column_to_rownames(var="patient_ID")
head(valid_label)

                       class batch
MP2PRT_ALL_1056          ALL    38
GSM2433722               CLL     4
GSM1554597               AML     1
MP2PRT_ALL_60            ALL    38
GSM7461803      non-leukemia    29
GSM3104740               CLL    10


In [ ]:
%%R

all(colnames(normalized_val) %in% rownames(valid_label))

[1] TRUE


In [ ]:
%%R

all(colnames(normalized_val) == rownames(valid_label))

[1] TRUE


In [ ]:
%%R
ncol(normalized_val)

[1] 523


In [ ]:
%%R
#write.table(normalized_val, file = "data_validation_normalized.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

In [ ]:
%%R
check_val <- read_tsv("data_validation_normalized.tsv")
head(check_val)

New names:
• `` -> `...1`
Rows: 76765 Columns: 524
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): ...1
dbl (523): MP2PRT_ALL_1056, GSM2433722, GSM1554597, MP2PRT_ALL_60, GSM746180...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 524
  ...1      MP2PRT_ALL_1056 GSM2433722 GSM1554597 MP2PRT_ALL_60 GSM7461803
  <chr>               <dbl>      <dbl>      <dbl>         <dbl>      <dbl>
1 5_8S_rRNA             0          0         0             0           0  
2 7SK                   0          0         0             0           0  
3 A1BG                291.       113.       94.1         161.         30.5
4 A1BG-AS1           1049.       833.     1208.          140.        192. 
5 A1CF                 13.2        0         3.06          4.74        0  
6 A2M                 128.        44.0    1743.         

### Testing Data

In [ ]:
%%R
df_test <- read_tsv("testing_dataset.tsv")
head(df_test)

Rows: 76765 Columns: 264
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): patient_ID
dbl (263): MP2PRT_ALL_1208, GSM4280607, MP2PRT_ALL_969, GSM4280595, GSM74618...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 264
  patient_ID MP2PRT_ALL_1208 GSM4280607 MP2PRT_ALL_969 GSM4280595 GSM7461819
  <chr>                <dbl>      <dbl>          <dbl>      <dbl>      <dbl>
1 5_8S_rRNA                0          0              0          0          0
2 7SK                      0          0              0          0          0
3 A1BG                    93         25             78         11         68
4 A1BG-AS1               300         40            266         23         26
5 A1CF                     2          0              2          1          0
6 A2M                   1323         52           1546        

In [ ]:
%%R
test_count <- df_test %>% remove_rownames %>% column_to_rownames(var="patient_ID")
head(test_count)

          MP2PRT_ALL_1208 GSM4280607 MP2PRT_ALL_969 GSM4280595 GSM7461819
5_8S_rRNA               0          0              0          0          0
7SK                     0          0              0          0          0
A1BG                   93         25             78         11         68
A1BG-AS1              300         40            266         23         26
A1CF                    2          0              2          1          0
A2M                  1323         52           1546         37         40
          NBM13_CD34 MP2PRT_ALL_52 GSM3039814 MP2PRT_ALL_1113 MP2PRT_ALL_886
5_8S_rRNA          0             0          0               0              0
7SK                0             0          0               0              0
A1BG              14            35         58             748             68
A1BG-AS1          48            71         67             811            261
A1CF               0             4         27               1              4
A2M              119

In [ ]:
%%R
normalized_test <- apply_normalization(test_count, train_size_factors)

In addition: Warning message:
In t(new_counts)/sf_reference :
  longer object length is not a multiple of shorter object length


In [ ]:
%%R
head(normalized_test)

          MP2PRT_ALL_1208 GSM4280607 MP2PRT_ALL_969 GSM4280595 GSM7461819
5_8S_rRNA        0.000000    0.00000        0.00000  0.0000000    0.00000
7SK              0.000000    0.00000        0.00000  0.0000000    0.00000
A1BG           323.560030   31.28971       64.45417 22.4813579  103.86023
A1BG-AS1      1920.283252   26.72338     1019.88062 43.1604997   10.12201
A1CF             2.501018    0.00000        1.32561  0.6075146    0.00000
A2M            868.396230   38.95024     1502.02094 31.8829534   32.81947
          NBM13_CD34 MP2PRT_ALL_52 GSM3039814 MP2PRT_ALL_1113 MP2PRT_ALL_886
5_8S_rRNA    0.00000      0.000000   0.000000       0.0000000       0.000000
7SK          0.00000      0.000000   0.000000       0.0000000       0.000000
A1BG        11.06348     21.149979  88.109216    4732.9054706      32.913925
A1BG-AS1    29.87506     80.657126  33.516131     526.4064046     247.665052
A1CF         0.00000      2.536998  27.707848       0.9437982       6.404738
A2M        213.10436

In [ ]:
%%R
l_test <- read_tsv("testing_labels.tsv")
head(l_test)

Rows: 263 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): patient_ID, class
dbl (1): batch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 3
  patient_ID      class        batch
  <chr>           <chr>        <dbl>
1 MP2PRT_ALL_1208 ALL             38
2 GSM4280607      CML             16
3 MP2PRT_ALL_969  ALL             38
4 GSM4280595      CML             16
5 GSM7461819      non-leukemia    29
6 NBM13_CD34      non-leukemia    34


In [ ]:
%%R
test_label <- l_test %>% remove_rownames %>% column_to_rownames(var="patient_ID")
head(test_label)

                       class batch
MP2PRT_ALL_1208          ALL    38
GSM4280607               CML    16
MP2PRT_ALL_969           ALL    38
GSM4280595               CML    16
GSM7461819      non-leukemia    29
NBM13_CD34      non-leukemia    34


In [ ]:
%%R

all(colnames(normalized_test) %in% rownames(test_label))

[1] TRUE


In [ ]:
%%R

all(colnames(normalized_test) == rownames(test_label))

[1] TRUE


In [ ]:
%%R
ncol(normalized_test)

[1] 263


In [ ]:
%%R
#write.table(normalized_test, file = "data_testing_normalized.tsv", sep = "\t", col.names = NA,
 #           qmethod = "double")

In [ ]:
%%R
check_test <- read_tsv("data_testing_normalized.tsv")
head(check_test)

New names:
• `` -> `...1`
Rows: 76765 Columns: 264
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (1): ...1
dbl (263): MP2PRT_ALL_1208, GSM4280607, MP2PRT_ALL_969, GSM4280595, GSM74618...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 264
  ...1      MP2PRT_ALL_1208 GSM4280607 MP2PRT_ALL_969 GSM4280595 GSM7461819
  <chr>               <dbl>      <dbl>          <dbl>      <dbl>      <dbl>
1 5_8S_rRNA            0           0             0         0            0  
2 7SK                  0           0             0         0            0  
3 A1BG               324.         31.3          64.5      22.5        104. 
4 A1BG-AS1          1920.         26.7        1020.       43.2         10.1
5 A1CF                 2.50        0             1.33      0.608        0  
6 A2M                868.         39.0        150